In [5]:
pip install fastapi uvicorn joblib numpy pydantic nest_asyncio


[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [7]:
import pandas as pd
import joblib
import numpy as np
from fastapi import FastAPI, HTTPException
from pydantic import BaseModel
from typing import List
import uvicorn
import nest_asyncio
import threading
nest_asyncio.apply()

### Bước 1: Chuẩn bị và kiểm tra dữ liệu đầu vào từ dataset

"Chuẩn bị": Đoạn mã đọc file CSV, lấy một dòng dữ liệu (dòng 100,000), và xử lý dữ liệu (loại bỏ cột Label, chuyển thành danh sách, xử lý NaN).

"Kiểm tra": Mã in ra các thông tin để kiểm tra dữ liệu (số lượng đặc trưng, có NaN không, nhãn thực tế).

"Dữ liệu đầu vào từ dataset": Dữ liệu được lấy từ file CSV (processed_cse_cic_ids2018_malicious_benign.csv) và mục đích là chuẩn bị đầu vào cho API dự đoán.

In [10]:
# Đọc file dữ liệu
try:
    data = pd.read_csv('Dataset_To_Train_Models/processed_cse_cic_ids2018_malicious_benign.csv')
except FileNotFoundError:
    print("Lỗi: File CSV không tồn tại. Vui lòng kiểm tra đường dẫn 'Dataset_To_Train_Models/processed_cse_cic_ids2018_malicious_benign.csv'.")
    raise

# Lấy dòng thứ [tùy chọn]
new_data_row = data.iloc[800000]

# Loại bỏ cột Label và chuyển thành danh sách
features = new_data_row.drop('Label').tolist()

# Kiểm tra dữ liệu
print("Số lượng đặc trưng:", len(features))
print("Dữ liệu đặc trưng:", features)
print("Có giá trị NaN hoặc None không:", any(pd.isna(x) for x in features))
print("Nhãn thực tế:", "Benign" if new_data_row['Label'] == 0 else "Malicious")

# Xử lý NaN hoặc None nếu có
#features = [0 if pd.isna(x) else float(x) for x in features]
#features = [int(x) if isinstance(x, bool) else x for x in features]
features = [0.0 if pd.isna(x) else float(x) for x in features]
# Kiểm tra lại sau khi xử lý
print("Dữ liệu đặc trưng sau khi xử lý:", features)
print("Có giá trị NaN hoặc None sau khi xử lý không:", any(pd.isna(x) for x in features))

Số lượng đặc trưng: 72
Dữ liệu đặc trưng: [5282937, 5, 3, 935, 370.0, 935, 0, 187.0, 418.1447118, 370, 0, 123.3333333, 213.6195996, 754705.2857, 1975318.312, 5234232.0, 2.0, 5282937.0, 1320734.25, 2609028.472, 5234239.0, 27.0, 23291.0, 11645.5, 15962.93559, 22933.0, 358.0, 168, 104, 0.946443238, 0.567865943, 0, 935, 145.0, 320.5269099, 102737.5, 0, 0, 0, 0, 1, 0, 0, 0, 0, 163.125, 187.0, 123.3333333, 0, 0, 0, 0, 0, 0, 5, 935, 3, 370, 219, 211, 1, 32, 0.0, 0.0, 0.0, 0.0, 5234232.0, 0.0, 5234232.0, 5234232.0, False, True]
Có giá trị NaN hoặc None không: False
Nhãn thực tế: Benign
Dữ liệu đặc trưng sau khi xử lý: [5282937.0, 5.0, 3.0, 935.0, 370.0, 935.0, 0.0, 187.0, 418.1447118, 370.0, 0.0, 123.3333333, 213.6195996, 754705.2857, 1975318.312, 5234232.0, 2.0, 5282937.0, 1320734.25, 2609028.472, 5234239.0, 27.0, 23291.0, 11645.5, 15962.93559, 22933.0, 358.0, 168.0, 104.0, 0.946443238, 0.567865943, 0.0, 935.0, 145.0, 320.5269099, 102737.5, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 163.125

### Bước 2: Thiết lập và triển khai API FastAPI cho dự đoán

"Thiết lập": Đoạn mã khởi tạo FastAPI, tải mô hình và scaler, định nghĩa cấu trúc dữ liệu đầu vào (InputData), và thiết lập các endpoint (/ và /predict).

"Triển khai": Mã chạy API bằng Uvicorn trên http://127.0.0.1:8000.

"API FastAPI": Đây là trọng tâm của đoạn mã, sử dụng FastAPI để tạo API.

"Cho dự đoán": Mục đích chính của API là dự đoán (/predict) dựa trên mô hình Random Forest.

In [12]:
# Áp dụng nest_asyncio để chạy trong Jupyter Notebook
nest_asyncio.apply()

# Khởi tạo FastAPI
app = FastAPI(title="Random Forest Prediction API")

# Tải mô hình và scaler
try:
    model = joblib.load('trained_models/random-forest-classifier_model.pkl')
    scaler = joblib.load('trained_models/rf_scaler.pkl')
    print("Mô hình và scaler đã được tải thành công.")
    print("Số đặc trưng mong đợi:", scaler.n_features_in_)
except FileNotFoundError as e:
    print(f"Lỗi: {e}. Vui lòng kiểm tra file mô hình và scaler trong thư mục 'trained_models'.")
    raise

# Định nghĩa cấu trúc dữ liệu đầu vào
class InputData(BaseModel):
    features: List[float]

class PredictionResponse(BaseModel):
    prediction: str
    probability: dict

# Endpoint kiểm tra API
@app.get("/")
async def root():
    return {"message": "Welcome to the Random Forest Prediction API"}

# Endpoint dự đoán
@app.post(
    "/predict",
    response_model=PredictionResponse,
    summary="Dự đoán loại truy cập Web",
    description="Trả về nhãn 'Benign' hoặc 'Malicious' cùng với xác suất."
)
async def predict(data: InputData):
    try:
        features = np.array(data.features).reshape(1, -1)
        expected_features = scaler.n_features_in_
        if features.shape[1] != expected_features:
            raise HTTPException(
                status_code=400,
                detail=f"Số lượng đặc trưng không đúng. Mong đợi {expected_features}, nhận được {features.shape[1]}."
            )
        features_scaled = scaler.transform(features)
        prediction = model.predict(features_scaled)[0]
        probability = model.predict_proba(features_scaled)[0].tolist()
        label = "Benign" if prediction == 0 else "Malicious"
        return {
            "prediction": label,
            "probability": {
                "Benign": probability[0],
                "Malicious": probability[1]
            }
        }
    except Exception as e:
        raise HTTPException(status_code=500, detail=f"Lỗi trong quá trình dự đoán: {str(e)}")

Mô hình và scaler đã được tải thành công.
Số đặc trưng mong đợi: 72


### Bước 3: Chạy FaskAPI

In [23]:
def run():
    uvicorn.run(app, host="127.0.0.1", port=8000)

thread = threading.Thread(target=run)
thread.start()

INFO:     Started server process [11776]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
ERROR:    [Errno 10048] error while attempting to bind on address ('127.0.0.1', 8000): [winerror 10048] only one usage of each socket address (protocol/network address/port) is normally permitted
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.


### Dùng trình duyệt để xem tài liệu API và thử nghiệm:

Truy cập: http://127.0.0.1:8000/docs

Bạn sẽ thấy giao diện Swagger UI với danh sách các endpoint của API:

/ (GET): Endpoint chào mừng.

/predict (POST): Endpoint dự đoán.

Tìm endpoint /predict, nhấn "Try it out".

Dán dữ liệu JSON vào:

In [ ]:
Benign:
{
  "features": [5282937.0, 5.0, 3.0, 935.0, 370.0, 935.0, 0.0, 187.0, 418.1447118, 370.0, 0.0, 123.3333333, 213.6195996, 754705.2857, 1975318.312, 5234232.0, 2.0, 5282937.0, 1320734.25, 2609028.472, 5234239.0, 27.0, 23291.0, 11645.5, 15962.93559, 22933.0, 358.0, 168.0, 104.0, 0.946443238, 0.567865943, 0.0, 935.0, 145.0, 320.5269099, 102737.5, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 163.125, 187.0, 123.3333333, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 5.0, 935.0, 3.0, 370.0, 219.0, 211.0, 1.0, 32.0, 0.0, 0.0, 0.0, 0.0, 5234232.0, 0.0, 5234232.0, 5234232.0, 0.0, 1.0]
}

Malicious:
{
  "features": [19181.0, 2.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 19181.0, 0.0, 19181.0, 19181.0, 19181.0, 19181.0, 0.0, 19181.0, 19181.0, 0.0, 0.0, 0.0, 0.0, 0.0, 64.0, 0.0, 104.2698504, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 2.0, 0.0, 0.0, 0.0, 225.0, -1.0, 0.0, 32.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0]
}


### Đánh giá

Thành công:
Endpoint /predict đã hoạt động đúng với yêu cầu POST đầu tiên (Code 200).
Dự đoán chính xác: Benign, khớp với nhãn thực tế.
Xác suất rất cao (99.98% cho Benign), cho thấy mô hình hoạt động tốt với dữ liệu này.

### Kết quả lỗi (Code 422)
Kết quả trả về (Code 422 - Validation Error):

{
  
  "detail": [
   
    {
      
      "loc": ["body", 0],
      
      "msg": "value is not a valid dict",
      
      "type": "type_error.dict"
    
    }
  
  ]

}


Lỗi 422: Lỗi này không ảnh hưởng đến kết quả chính, chỉ xảy ra khi bạn thử gửi dữ liệu không đúng định dạng. Để tránh lỗi này, luôn đảm bảo dữ liệu gửi đi đúng cấu trúc {"features": [danh_sách_72_số]}.

### Kiểm tra số dòng

In [17]:
# Đọc file dữ liệu (thay 'your_dataset.csv' bằng tên file của bạn)
df = pd.read_csv('Dataset_To_Train_Models/processed_cse_cic_ids2018_malicious_benign.csv')

# Kiểm tra tổng số dòng
total_rows = len(df)
print(f"Tổng số dòng trong tập dữ liệu: {total_rows}")

# (Tùy chọn) Xem thông tin tổng quan về DataFrame
df.info()

Tổng số dòng trong tập dữ liệu: 4772331
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4772331 entries, 0 to 4772330
Data columns (total 73 columns):
 #   Column             Dtype  
---  ------             -----  
 0   Flow Duration      int64  
 1   Tot Fwd Pkts       int64  
 2   Tot Bwd Pkts       int64  
 3   TotLen Fwd Pkts    int64  
 4   TotLen Bwd Pkts    float64
 5   Fwd Pkt Len Max    int64  
 6   Fwd Pkt Len Min    int64  
 7   Fwd Pkt Len Mean   float64
 8   Fwd Pkt Len Std    float64
 9   Bwd Pkt Len Max    int64  
 10  Bwd Pkt Len Min    int64  
 11  Bwd Pkt Len Mean   float64
 12  Bwd Pkt Len Std    float64
 13  Flow IAT Mean      float64
 14  Flow IAT Std       float64
 15  Flow IAT Max       float64
 16  Flow IAT Min       float64
 17  Fwd IAT Tot        float64
 18  Fwd IAT Mean       float64
 19  Fwd IAT Std        float64
 20  Fwd IAT Max        float64
 21  Fwd IAT Min        float64
 22  Bwd IAT Tot        float64
 23  Bwd IAT Mean       float64
 24  Bwd IAT St

In [27]:
# Chọn dòng cụ thể (ví dụ: dòng 800,000)
row_number = 800000

# Kiểm tra xem dòng có tồn tại không
if row_number < total_rows:
    selected_row = df.iloc[row_number]
    print(f"Dữ liệu tại dòng {row_number}:")
    print(selected_row)
    # Nếu bạn có cột nhãn (label), ví dụ 'label'
    if 'label' in df.columns:
        label = selected_row['label']
        print(f"Nhãn của dòng {row_number}: {label}")
else:
    print(f"Dòng {row_number} không tồn tại. Tổng số dòng chỉ có {total_rows}.")

Dữ liệu tại dòng 800000:
Flow Duration        5282937
Tot Fwd Pkts               5
Tot Bwd Pkts               3
TotLen Fwd Pkts          935
TotLen Bwd Pkts        370.0
                     ...    
Idle Max           5234232.0
Idle Min           5234232.0
Protocol_17            False
Protocol_6              True
Label                      0
Name: 800000, Length: 73, dtype: object
